# Notebook to clean RTS output polygons

## Issue

* intersection with Water, bare rocks

## Solution

* intersect polygons with water datasets
  * Land cover?
    * ESRI
    * Other?
  * dedicated water datasets?

## Imports

In [ ]:
import geemap
import geopandas as gpd
from pathlib import Path
import ee
ee.Initialize()

## Data Loading

In [ ]:
# define file
input_vector = Path('../data/input/SePe.gpkg')
#input_vector = Path('../data/input/ETaymyr.gpkg')

# load file to gdf
gdf = gpd.read_file(input_vector)

# apply filter
out = filter_remove_water(gdf, threshold=0.2)

# save to output
out.to_file('../data/output/ETaymyr_filtered.gpkg')

In [ ]:
def filter_remove_water(gdf, threshold=0.2):
    """
    Filter a GeoDataFrame to remove features with high water coverage based on ESRI Global Land Use Land Cover data.

    This function converts the input GeoDataFrame to an Earth Engine FeatureCollection,
    uses ESRI's Global LULC 10m dataset to create a binary water mask, calculates the
    mean water coverage for each feature, and filters out features exceeding the specified
    water coverage threshold.

    Parameters:
    -----------
    gdf : geopandas.GeoDataFrame
        Input GeoDataFrame containing the features to be filtered.
    threshold : float, optional
        The maximum allowable proportion of water coverage for a feature to be retained.
        Features with mean water coverage exceeding this threshold will be removed.
        Default is 0.2 (20% water coverage).

    Returns:
    --------
    geopandas.GeoDataFrame
        A filtered GeoDataFrame containing only the features with water coverage
        below or equal to the specified threshold.

    Notes:
    ------
    - This function requires Earth Engine to be initialized.
    - It uses the ESRI Global Land Use Land Cover 10m dataset from Earth Engine.
    - The water mask is created by identifying pixels with a value of 1 in the LULC dataset.
    - The function assumes a scale of 10 meters for calculations, matching the LULC dataset resolution.

    Example:
    --------
    >>> import geopandas as gpd
    >>> import ee
    >>> ee.Initialize()
    >>> gdf = gpd.read_file('path/to/your/shapefile.shp')
    >>> filtered_gdf = filter_remove_water(gdf, threshold=0.3)
    """
    
    # convert gdf to ee FC
    rts_ee = geemap.gdf_to_ee(gdf)
    # load gee layer
    esri_lulc2020 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m")
    # filter to rts footprint
    filtered = esri_lulc2020.filterBounds(rts_ee)
    # get binary water mask
    water_layer = filtered.mosaic().eq(1).unmask()
    # reduce regions and get value
    reduced = ee.Image.reduceRegions(water_layer, rts_ee, reducer=ee.Reducer.mean(), scale=10)
    # convert to gdf
    gdf_out = geemap.ee_to_gdf(reduced)
    # filter to no water
    gdf_filtered = gdf.loc[gdf_out.query(f'mean <= {threshold}').index]
    
    return gdf_filtered
